In [1]:
import numpy as np
import torch
import pyepo

Auto-Sklearn cannot be imported.


## Data Set and Optimization Solver

In [2]:
# generate data
num_node = 50 # node size
num_data = 1000 # number of training data
num_feat = 10 # size of feature
deg = 4 # polynomial degree
e = 0.5 # noise width
feats, costs = pyepo.data.tsp.genData(num_data, num_feat, num_node, deg, e, seed=42)

In [3]:
from model import tspDFJModel
# set solver
optmodel = tspDFJModel(num_node)
# test
optmodel.setObj(costs[0])
sol, obj = optmodel.solve()
print("Obj: {}".format(obj))
tour = optmodel.getTour(sol)
print(" -> ".join(map(str, tour)))

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-01
Obj: 83.2908
0 -> 5 -> 3 -> 41 -> 12 -> 20 -> 23 -> 49 -> 8 -> 22 -> 9 -> 24 -> 44 -> 14 -> 6 -> 19 -> 46 -> 39 -> 17 -> 37 -> 4 -> 32 -> 34 -> 45 -> 43 -> 26 -> 35 -> 33 -> 28 -> 38 -> 1 -> 40 -> 48 -> 29 -> 15 -> 10 -> 25 -> 30 -> 47 -> 11 -> 27 -> 2 -> 36 -> 18 -> 7 -> 13 -> 31 -> 42 -> 21 -> 16 -> 0


In [4]:
from dataset import optDatasetConstrs
dataset_train = optDatasetConstrs(optmodel, feats, costs=costs) # with binding constr

Optimizing for optDataset...


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [04:16<00:00,  3.89it/s]


In [5]:
# get data loader
from torch.utils.data import DataLoader
from dataset import collate_fn
batch_size = 32
loader_train = DataLoader(dataset_train, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)

## Prediction Model

In [6]:
import torch
from torch import nn

# build linear model
class LinearRegression(nn.Module):

    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(num_feat, num_node*(num_node-1)//2)

    def forward(self, x):
        out = self.linear(x)
        return out

## Train

In [7]:
lr = 5e-2
num_epochs = 1

In [8]:
import time
from tqdm import tqdm

def pipeline(reg, forward_func, loss_func, lr, num_epochs, loader_train, seed=42):
    # set random seed
    np.random.seed(42)
    torch.manual_seed(42)
    # set optimizer
    optimizer = torch.optim.Adam(reg.parameters(), lr=lr)
    # running time
    elapsed = 0
    for epoch in range(num_epochs):
        # training
        tbar = tqdm(loader_train)
        tick = time.time()
        for data in tbar:
            # forward pass)
            loss = forward_func(data, reg, loss_func)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            tbar.set_description("Loss: {:8.4f}".format(loss.item()))
        # record time
        tock = time.time()
        elapsed += tock - tick
    time.sleep(1)
    print("Evaluation:")
    print("Training Elapsed Time: {:.2f} Sec".format(elapsed))

def forwardCAVE(data, reg, loss_func):
    # unzip data
    x, _, _, t_ctr = data
    # predict
    cp = reg(x)
    # loss
    loss = loss_func(cp, t_ctr)
    return loss

### Loss 

In [9]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import torch

from func import abstractConeAlignedCosine

class exactConeAlignedCosine(abstractConeAlignedCosine):
    """
    A autograd module to align cone and vector with exact cosine similarity loss
    """
    def __init__(self, optmodel, warmstart=True, conecheck=False):
        """
        Args:
            optmodel (optModel): an PyEPO optimization model
            warmstart (bool): start QP with initial solutions or not
            conecheck (bool): check if cost vector is in the cone or not
        """
        super().__init__(optmodel)
        self.warmstart = warmstart
        self.conecheck = conecheck

    def _getProjection(self, pred_cost, tight_ctrs):
        """
        A method to get the projection of the vector onto the polar cone via solving a quadratic programming
        """
        # get device
        device = pred_cost.device
        # to numpy
        pred_cost = pred_cost.detach().cpu().numpy()
        tight_ctrs = tight_ctrs.detach().cpu().numpy()
        # init loss
        proj = torch.empty(pred_cost.shape).to(device)
        # calculate projection
        for i, (cp, ctr) in enumerate(zip(pred_cost, tight_ctrs)):
            proj[i] = self._solveQP(cp, ctr, self.warmstart, self.conecheck)
        # normalize
        proj = proj / proj.norm(dim=1, keepdim=True)
        return proj

    @staticmethod
    def _solveQP(cp, ctr, warmstart, conecheck):
        """
        A static method to solve quadratic programming.
        """
        # drop pads
        ctr = ctr[np.abs(ctr).sum(axis=1) > 1e-7]
        # check if in the cone
        if conecheck and _checkInCone(cp, ctr):
            return torch.FloatTensor(cp.copy())
        # ceate a model
        m = gp.Model("projection")
        # turn off output
        m.Params.outputFlag = 0
        # numerical precision
        m.Params.FeasibilityTol = 1e-3
        m.Params.OptimalityTol = 1e-3
        # focus on numeric problem
        m.Params.NumericFocus = 3
        # varibles
        λ = m.addMVar(len(ctr), name="λ")
        # warm-start
        if warmstart:
            init_λ = np.zeros(len(ctr))
            sign = ctr[-len(cp):].sum(axis=0)
            init_λ[-len(cp):] = np.abs(cp) * (np.sign(cp) == np.sign(sign)) # hyperqudrants
            λ.Start = init_λ
        # objective function
        obj = (cp - λ @ ctr) @ (cp - λ @ ctr)
        m.setObjective(obj, GRB.MINIMIZE)
        # solve
        m.optimize()
        # get solutions
        p = np.array(λ.X) @ ctr
        # normalize
        p = torch.FloatTensor(p / np.linalg.norm(p))
        return p
    
def _checkInCone(cp, ctr):
    """
    Method to check if the given cost vector in the cone
    """
    # ceate a model
    m = gp.Model("Cone Combination")
    # turn off output
    m.Params.outputFlag = 0
    # numerical precision
    m.Params.FeasibilityTol = 1e-3
    m.Params.OptimalityTol = 1e-3
    # varibles
    λ = m.addMVar(len(ctr), name="λ")
    # constraints
    m.addConstr(λ @ ctr == cp)
    # objective function
    m.setObjective(0, GRB.MINIMIZE)
    # solve the model
    m.optimize()
    # return the status of the model
    return m.status == GRB.OPTIMAL

(CVXPY) Nov 22 02:10:48 AM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')


### Standard

In [10]:
# init model
reg = LinearRegression()

In [11]:
# init loss
ca_cos = exactConeAlignedCosine(optmodel, warmstart=False, conecheck=False)

In [12]:
pipeline(reg, forwardCAVE, ca_cos, lr, num_epochs, loader_train)

Loss:  -0.9979: 100%|██████████████████████████████████████████████████████████████████| 32/32 [03:32<00:00,  6.65s/it]


Evaluation:
Training Elapsed Time: 212.92 Sec


### Warm Starting 

In [13]:
# init model
reg = LinearRegression()

In [14]:
# init loss
ca_cos = exactConeAlignedCosine(optmodel, warmstart=True, conecheck=False)

In [15]:
pipeline(reg, forwardCAVE, ca_cos, lr, num_epochs, loader_train)

Loss:  -0.9982: 100%|██████████████████████████████████████████████████████████████████| 32/32 [03:30<00:00,  6.57s/it]


Evaluation:
Training Elapsed Time: 210.24 Sec


### Cone Check 

In [16]:
# init model
reg = LinearRegression()

In [17]:
# init loss
ca_cos = exactConeAlignedCosine(optmodel, warmstart=False, conecheck=True)

In [18]:
pipeline(reg, forwardCAVE, ca_cos, lr, num_epochs, loader_train)

Loss:  -0.9982: 100%|██████████████████████████████████████████████████████████████████| 32/32 [03:57<00:00,  7.42s/it]


Evaluation:
Training Elapsed Time: 237.42 Sec
